### Importing necessary stuff !

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

   ### Reading in data (and defining necessary constants ) !

In [ ]:
full_data = pd.read_csv('../input/train.csv')

In [ ]:
LABEL = 'label'

In [ ]:
NUM_PREDICTOR_COLS = 784
PREDICTOR_COLS = ['pixel' + str(i) for i in range(NUM_PREDICTOR_COLS)]

In [ ]:
REL_COLUMNS = PREDICTOR_COLS.copy()
REL_COLUMNS.append('label')

### Checking for null values.

In [ ]:
full_data.isnull().values.any()

### Split into training and validation sets

In [ ]:
LEN_TRAIN_SET = int(0.8 * len(full_data))
train_data = full_data[0:LEN_TRAIN_SET]
validation_data = full_data[LEN_TRAIN_SET:len(full_data)]

In [ ]:
assert(len(train_data) + len(validation_data) == len(full_data))

In [ ]:
len(full_data)

In [ ]:
len(train_data)

In [ ]:
len(validation_data) 

### Fit model on training data


In [ ]:
def make_train_input_fn(df, num_epochs):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df[LABEL],
        shuffle=True,
        num_epochs=num_epochs)

In [ ]:
def make_eval_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=df[LABEL],
        shuffle=False)

In [ ]:
def make_prediction_input_fn(df):
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=None,
        shuffle=False)    

In [ ]:
def make_features():
    input_cols = [tf.feature_column.numeric_column(col) for col in PREDICTOR_COLS]
    return input_cols

In [ ]:
X_train = train_data[PREDICTOR_COLS]
Y_train = np.ravel(train_data[['label']])

In [ ]:
OUTDIR = 'classification_outputs'
import shutil
shutil.rmtree(OUTDIR, ignore_errors=True)

In [ ]:
N_CLASSES = 10

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=make_features(),
                                       n_classes=N_CLASSES,
                                       model_dir=OUTDIR)


In [ ]:
model.train(input_fn=make_train_input_fn(train_data[REL_COLUMNS], num_epochs=100))    

In [ ]:
metrics = model.evaluate(input_fn = make_eval_input_fn(train_data[REL_COLUMNS]))
print(metrics['accuracy'])

In [ ]:
metrics = model.evaluate(input_fn = make_eval_input_fn(validation_data[REL_COLUMNS]))
print(metrics['accuracy'])

In [ ]:
# Results
# Linear Classifier using all components(epoch of 1). Out of sample score : 0.899
# Linear Classifier using all components(epoch of 100). Out of sample score : 0.8761
# Linear Classifier using PCA (200) components(epoch of 100). Out of sample score : 0.877
# Linear Classifier using PCA (300) components(epoch of 100). Out of sample score : 0.876
# Linear Classifier using PCA (300) components(epoch of 1). Out of sample score : 0.8864



### Do PCA here

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
NUM_PCA_COMPONENTS = 300
pca_input = PCA(n_components=NUM_PCA_COMPONENTS)
pca_input.fit(train_data[PREDICTOR_COLS])

In [ ]:
PCA_COLS = ['pca' + str(i) for i in range(NUM_PCA_COMPONENTS)]
REL_PCA_COLUMNS = PCA_COLS.copy()
REL_PCA_COLUMNS.append('label')

In [ ]:
transformed_train = pca_input.transform(train_data[PREDICTOR_COLS])
transformed_validation = pca_input.transform(validation_data[PREDICTOR_COLS])

transformed_train_df = pd.DataFrame(transformed_train, columns=PCA_COLS)
transformed_validation_df = pd.DataFrame(transformed_validation, columns=PCA_COLS)

transformed_train_df[LABEL] = train_data[LABEL].values
transformed_validation_df[LABEL] = validation_data[LABEL].values

In [ ]:
OUTDIR = 'classification_outputs_pca'
import shutil
shutil.rmtree(OUTDIR, ignore_errors=True)

In [ ]:
model_pca = tf.estimator.LinearClassifier(
    feature_columns=[tf.feature_column.numeric_column(col) for col in PCA_COLS],
    n_classes=N_CLASSES,
    model_dir=OUTDIR)


In [ ]:
model_pca.train(input_fn=make_train_input_fn(transformed_train_df[REL_PCA_COLUMNS], num_epochs=1))    

In [ ]:
metrics = model_pca.evaluate(input_fn = make_eval_input_fn(transformed_train_df[REL_PCA_COLUMNS]))
print(metrics['accuracy'])

In [ ]:
metrics = model_pca.evaluate(input_fn = make_eval_input_fn(transformed_validation_df[REL_PCA_COLUMNS]))
print(metrics['accuracy'])

In [ ]:
def get_train_test_data(train_data,
                        X_columns=PREDICTOR_COLS, 
                        Y_columns=['label']):
    # Simple training and testing
    X = train_data[X_columns]
    Y = train_data[Y_columns]
 
    len_train_set = int(0.8 * len(train_data))
    X_train = X[0:len_train_set]
    Y_train = Y[0:len_train_set]
    X_test = X[len_train_set:len(X)]
    Y_test = Y[len_train_set:len(Y)]
    rel_train_data = train_data[0:len_train_set]
    rel_test_data = train_data[len_train_set:len(train_data)]

    return (X_train, X_test, Y_train, Y_test, X, Y, rel_train_data, rel_test_data)


### Train final model on train data and make predictions on validation set.

In [ ]:
OUTDIR = 'classification_outputs_pca'
import shutil
shutil.rmtree(OUTDIR, ignore_errors=True)

In [ ]:
(X_train, X_validation, Y_train, Y_validation, X, Y, train_data, validation_data) = get_train_test_data(full_data)
model = tf.estimator.LinearClassifier(feature_columns=make_features(),
                                       n_classes=N_CLASSES,
                                       model_dir=OUTDIR)

# Fit the model
model.train(input_fn=make_train_input_fn(train_data[REL_COLUMNS], num_epochs=1))    

predictions_temp = model.predict(input_fn=make_prediction_input_fn(validation_data[PREDICTOR_COLS]))
predictions_validation = [int(i['classes'][0]) for i in predictions_temp]

# Make and dump predictions to a file.
Y_validation = Y_validation.assign(prediction=predictions_validation)
Y_validation['ImageId'] = Y_validation.index
Y_validation[['ImageId', 'label', 'prediction']].to_csv('validation_logistic_classification_tensorflow.csv', index=False)
print(np.sum(Y_validation['label'] == Y_validation['prediction']) * 100.0/(len(Y_validation)))

### Making predictions on test data
NOTE : I have not included the code for making predictions using the model built using PCA , as we are focussing only on validation score for now.

In [ ]:
test_data = pd.read_csv('../input/test.csv')

In [ ]:
OUTDIR = 'classification_outputs_pca'
import shutil
shutil.rmtree(OUTDIR, ignore_errors=True)

In [ ]:
model = tf.estimator.LinearClassifier(feature_columns=make_features(),
                                       n_classes=N_CLASSES,
                                       model_dir=OUTDIR)

# Fit the model
model.train(input_fn=make_train_input_fn(full_data[REL_COLUMNS], num_epochs=1))    


In [ ]:
test_predictions = model.predict(input_fn=make_prediction_input_fn(test_data[PREDICTOR_COLS]))

In [ ]:
test_data['Label'] = [int(i['classes'][0]) for i in test_predictions]

In [ ]:
test_data['ImageId'] = np.arange(1, len(test_data)+1)

In [ ]:
test_data[['ImageId', 'Label']].to_csv('submission_logistic_classification_tensorflow.csv', index=False)